<a href="https://colab.research.google.com/github/Ashail33/textclustering/blob/main/UserAuth%20and%20payment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install firebase-admin
# Install necessary packages
!pip install streamlit stripe firebase-admin streamlit-jupyter
!npm install -g localtunnel



In [26]:
# # Install necessary packages
# !pip install streamlit stripe firebase-admin streamlit-jupyter
# !npm install -g localtunnel


# Define user data dictionary
userdata = {
    'stripe': userdata.get('stripe')  # Replace with your actual Stripe secret key
}

# Create the .streamlit directory and secrets.toml file
import os

os.makedirs("/root/.streamlit", exist_ok=True)

secrets_content = """
[general]
YOUR_DOMAIN = "http://localhost:8501"
"""

with open("/root/.streamlit/secrets.toml", "w") as secrets_file:
    secrets_file.write(secrets_content)

# Streamlit app code
app_code = f"""
import streamlit as st
import stripe
import firebase_admin
from firebase_admin import credentials, auth, firestore
import datetime
from datetime import timedelta

# Initialize Firebase
if not firebase_admin._apps:
    cred = credentials.Certificate("sticky.json")
    firebase_admin.initialize_app(cred)

db = firestore.client()

# Set your Stripe secret key
stripe.api_key = "{userdata['stripe']}"

# Define product prices
prices = {{
    "Option 1": 1.00,
    "Option 2": 2.00
}}

# Initialize session state variables
if "purchased" not in st.session_state:
    st.session_state.purchased = False
if "option" not in st.session_state:
    st.session_state.option = None
if "user" not in st.session_state:
    st.session_state.user = None
if "page" not in st.session_state:
    st.session_state.page = "home"

# Streamlit app
st.title("Streamlit App with Stripe Payments and Firebase")

# User authentication
def sign_up(email, password):
    try:
        user = auth.create_user(email=email, password=password)
        return user
    except Exception as e:
        st.error(f"An error occurred: {{e}}")
    return None

def log_in(email, password):
    try:
        user = auth.get_user_by_email(email)
        # This is a placeholder for verifying the password.
        # Firebase Admin SDK does not support password verification directly.
        # You would typically use Firebase Authentication client SDK on the client side for this.
        st.session_state.user = user
    except Exception as e:
        st.error(f"Invalid email or password: {{e}}")

# Home page with options to log in or sign up
if st.session_state.page == "home":
    st.header("Welcome")
    if st.session_state.user is None:
        if st.button("Go to Login"):
            st.session_state.page = "login"
        if st.button("Go to Sign Up"):
            st.session_state.page = "signup"
    else:
        st.session_state.page = "main"

# Login page
if st.session_state.page == "login":
    st.header("Login")
    login_email = st.text_input("Email", key="login_email")
    login_password = st.text_input("Password", type="password", key="login_password")
    if st.button("Login", key="login_submit"):
        log_in(login_email, login_password)
        if st.session_state.user:
            st.session_state.page = "main"
    if st.button("Back", key="login_back"):
        st.session_state.page = "home"

# Sign-up page
if st.session_state.page == "signup":
    st.header("Sign Up")
    signup_email = st.text_input("Email", key="signup_email")
    signup_password = st.text_input("Password", type="password", key="signup_password")
    if st.button("Sign Up", key="signup_submit"):
        user = sign_up(signup_email, signup_password)
        if user:
            st.success(f"User created: {{user.email}}")
            st.session_state.user = user
            st.session_state.page = "main"
    if st.button("Back", key="signup_back"):
        st.session_state.page = "home"

# Main page
if st.session_state.page == "main" and st.session_state.user:
    user = st.session_state.user
    st.success(f"Logged in as {{user.email}}")

    # Load user subscription data
    def get_user_subscription(user):
        doc_ref = db.collection("subscriptions").document(user.uid)
        doc = doc_ref.get()
        if doc.exists:
            return doc.to_dict()
        else:
            return None

    # Update user subscription data
    def update_user_subscription(user, option):
        doc_ref = db.collection("subscriptions").document(user.uid)
        subscription = get_user_subscription(user)
        if subscription:
            expiry_date = subscription.get("expiry_date")
            if expiry_date and datetime.datetime.strptime(expiry_date, "%Y-%m-%d") > datetime.datetime.now():
                new_expiry_date = datetime.datetime.strptime(expiry_date, "%Y-%m-%d") + timedelta(days=30)
            else:
                new_expiry_date = datetime.datetime.now() + timedelta(days=30)
        else:
            new_expiry_date = datetime.datetime.now() + timedelta(days=30)

        doc_ref.set({{
            "option": option,
            "expiry_date": new_expiry_date.strftime("%Y-%m-%d")
        }})

    # Display pricing options and handle payments
    st.header("Pricing Options")
    st.write("Choose your plan:")
    for option, price in prices.items():
        if st.button(f"Buy {{option}} for {{price}} Euro"):
            session = stripe.checkout.Session.create(
                payment_method_types=['card'],
                line_items=[{{
                    'price_data': {{
                        'currency': 'eur',
                        'product_data': {{
                            'name': option,
                        }},
                        'unit_amount': int(price * 100),
                    }},
                    'quantity': 1,
                }}],
                mode='payment',
                success_url=st.secrets["general"]["YOUR_DOMAIN"] + '?session_id={{CHECKOUT_SESSION_ID}}',
                cancel_url=st.secrets["general"]["YOUR_DOMAIN"],
            )
            st.session_state.purchased = True
            st.session_state.option = option
            st.write("Redirecting to payment...")
            st.write(f"[Click here to complete the payment]({{session.url}})", unsafe_allow_html=True)

    # After successful payment (mock success in demo)
    if st.session_state.purchased:
        update_user_subscription(st.session_state.user, st.session_state.option)
        st.success(f"You have purchased {{st.session_state.option}}")
        st.session_state.purchased = False

    # Display user subscriptions
    subscription = get_user_subscription(st.session_state.user)
    if subscription:
        expiry_date = datetime.datetime.strptime(subscription["expiry_date"], "%Y-%m-%d")
        if expiry_date > datetime.datetime.now():
            st.success(f"Your subscription for {{subscription['option']}} is valid until {{expiry_date.strftime('%Y-%m-%d')}}")
        else:
            st.warning("Your subscription has expired.")
    else:
        st.warning("You do not have any active subscriptions.")

# Remember to replace YOUR_DOMAIN with your actual domain in production
"""



# Write the content to app.py
with open("app.py", "w") as file:
    file.write(app_code)



In [22]:

# Write the content to app.py
with open("app.py", "w") as file:
    file.write(app_code)

# Step 3: Display the public IP address for localtunnel and run the apps
import subprocess
import urllib.request
import time
import streamlit_jupyter as stj

# Display the public IP address for localtunnel
public_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")
print("Password/Endpoint IP for localtunnel is:", public_ip)

# Function to get the localtunnel URL
def get_localtunnel_url():
    lt_process = subprocess.Popen(["npx", "localtunnel", "--port", "8501"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    while True:
        output = lt_process.stdout.readline().decode('utf-8').strip()
        if 'url' in output:
            print(output)
            break
        time.sleep(1)

# Run the Streamlit app in the background
streamlit_process = subprocess.Popen(["streamlit", "run", "app.py"])

# Get and display the localtunnel URL
get_localtunnel_url()


Password/Endpoint IP for localtunnel is: 34.170.97.149
your url is: https://bright-rings-run.loca.lt
